In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
def make_query(query, filepath='../database.sqlite'):
    """
    Execute a query on a SQLite database file and return the results as a pandas DataFrame.

    Parameters
    ----------
    query : str
        SQL query to be executed on the database file.
    filepath : str, optional
        Path to the database file, by default 'database.sqlite'.

    Returns
    -------
    pd.DataFrame
        A pandas DataFrame containing the results of the executed query.
    """
    with sqlite3.connect(filepath) as connection:
        cursor = connection.cursor()
        execution = cursor.execute(query)
        result = pd.DataFrame(execution.fetchall(), columns=[i[0] for i in execution.description])
        return result

In [3]:
def club_points_seasons(league='Premier'):
    """ Generates a DataFrame containing the total points earned by each club in each season of a given league.
    
    Args:
        league : str, optional 
            The league for which to generate the points table. Defaults to 'Premier'.
        
    Returns:
        pandas.core.frame.DataFrame
            The dataframe containing the total points earned by each club in each season of the specified season.
    """    
    container = make_query(
    f"""

    SELECT DISTINCT(season), name
    FROM Match
        INNER JOIN League
            ON Match.league_id = League.id
        WHERE League.name IN (SELECT name
                              FROM League
                              WHERE name LIKE '%{league}%'
                              LIMIT 1)
    """)
    league, season = container['name'][0], list(container['season'])
    for i in range(len(season)):
        current = make_query(
        f"""

        SELECT team_long_name AS Club, SUM(wins) AS '{season[i]}'
        FROM (SELECT CASE WHEN home_team_goal > away_team_goal THEN home_team_api_id 
                          WHEN home_team_goal < away_team_goal THEN away_team_api_id END AS winners,
                     (COUNT(*) * 3) AS wins

              FROM Match
                  INNER JOIN League
                      ON league_id = League.id
                  WHERE League.name LIKE '{league}' AND season='{season[i]}'

              GROUP BY winners

              UNION

              SELECT home_draw AS Team, SUM(draws) AS Draws
              FROM (SELECT CASE WHEN home_team_goal = away_team_goal THEN home_team_api_id END AS home_draw,
                           COUNT(*) AS draws

                    FROM Match
                        INNER JOIN League
                            ON league_id = League.id
                        WHERE League.name LIKE '{league}' AND season='{season[i]}'

                    GROUP BY home_draw

                    UNION

                    SELECT CASE WHEN home_team_goal = away_team_goal THEN away_team_api_id END AS away_draw,
                           COUNT(*)

                    FROM Match
                        INNER JOIN League
                            ON league_id = League.id
                        WHERE League.name LIKE '{league}' AND season='{season[i]}'

                    GROUP BY away_draw
                   )
              GROUP BY home_draw
              )
        INNER JOIN Team
            ON winners = Team.team_api_id

        GROUP BY winners

        """
        )
        if i == 0:
            df = current
        else:
            df = df.merge(right=current, on='Club', how='outer')
            df.fillna(0, inplace=True)
    df.set_index('Club', inplace=True)
    df = df.astype(int)
    return df

In [4]:
def rank(club='Manchester United', season='2008/2009'):
    """Retrieves the rank of a specific club in a specific season. 

    Parameters:
        club : str, optional
            The name of the club whose rank is to be retrieved. Defaults to 'Manchester United'.
        season : str, optional
            The season for which the rank is to be retrieved, in the format 'YYYY/YYYY'. Defaults to '2008/2009'.

    Returns:
        int
            The rank of the club in the season.
    """
    league, club = make_query(
    f"""

    SELECT league.name, team_long_name 

    FROM League
        INNER JOIN Match
    ON League.id = league_id
        INNER JOIN Team
    ON Match.home_team_api_id = Team.team_api_id

    WHERE team_long_name LIKE '%{club}%'
    
    LIMIT 1
    
    """).iloc[0, :]
    container = club_points_seasons(league)[season].sort_values(ascending=False).index
    for i in range(len(container)):
        if club == container[i]:
            return i + 1


In [5]:
club_points_seasons()

,2008/2009,2009/2010,2010/2011,2011/2012,2012/2013,2013/2014,2014/2015,2015/2016
Club,,,,,,,,
Chelsea,83,86,71,64,75,82,87,50
Manchester City,50,67,67,89,78,86,79,66
Portsmouth,41,28,0,0,0,0,0,0
Sunderland,36,44,47,45,39,38,38,33
Wigan Athletic,45,36,42,43,36,0,0,0
Middlesbrough,32,0,0,0,0,0,0,0
Bolton Wanderers,41,39,41,36,0,0,0,0
Tottenham Hotspur,51,70,62,69,72,66,64,70
Liverpool,86,63,58,52,61,84,62,60


In [6]:
rank('Manchester United', '2014/2015')

4